In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 12
### Team Member Names: Bill Bai, Soumik Debnath, Justin Yu
### Team Strategy Chosen: Risky (RISKY OR SAFE)

Below, we define start and end dates to get ticker data in order to create our portfolio. We defined it from 2020-11-26 to 2021-11-26, or a 1 year interval.

The reason why we chose to base our portfolio off of 1 year of historical data is because we wanted a timeframe that wasn't too long that our short-term risky portfolio would capture "too much" irrelevant data - for example, we chose to use standard deviation as a factor in creating our portfolio and using too long of a timeframe might cause our code to choose a stock that used to be volatile, but is now "safe". We also didn't want to use too short of a time interval since our portfolio uses correlation to choose stocks that tend to trade in the same direction - using too short of a timeframe would capture stocks that may be correlated for a short interval by pure coincidence or by broader-market forces.

Thus, the perfect interval we found after backtesting our code many times on different timeframes and lists of tickers happened to be 1 year. 1 year perfectly captures the stocks with the most momentum to make our portfolio risky, but also ensures that the stocks in our portfolio are correlated for a reason, rather than by pure coincidence or market forces.

Later on in our code, we'll also explain why we used daily data in calculating the risk metrics for our portfolio, rather than weekly or monthly data.

In [2]:
# Start and end date to base our portfolio tickers off of
start_date = "2020-11-26"
end_date = "2021-11-26"

# Portfolio starting date
comp_date = '2021-11-25'

Next we'll load the given list of tickers to choose our portfolio from.

In [3]:
# Read in tickers file and save as DF
tickers_path = "./Tickers.csv"
tickers_df = pd.read_csv(tickers_path, header=None).rename(columns={0: 'Ticker'})

In [4]:
# Display tickers dataframe
tickers_df

,Ticker
0,AAPL
1,ABBV
2,ABT
3,ACN
4,AGN
...,...
59,UNP
60,UPS
61,USB
62,RY.TO


Here, we create an info column in our tickers dataframe, where each cell contains the yf.Ticker(ticker).info dict for each ticker. We implemented this to increase efficiency in our code, since calling .info takes a few seconds for each call, and our list of tickers is very long. Thus, it is more efficient to save the info dict in memory rather than performing an API call each time we need a stock's info.

In [5]:
# Create an info cell for each ticker, saving time when filtering the tickers by volume and country.
info_column = {}

# Add info to dict
for idx, row in tickers_df.iterrows():
    ticker = row[0]  # Get ticker name
    ticker = yf.Ticker(ticker)
    info_column[idx] = ticker.info
    print('.', end='')  # So we can tell if the code is running

# Create column for the info]
tickers_df['Info'] = pd.Series(info_column)
tickers_df

................................................................

,Ticker,Info
0,AAPL,"{'zip': '95014', 'sector': 'Technology', 'full..."
1,ABBV,"{'zip': '60064-6400', 'sector': 'Healthcare', ..."
2,ABT,"{'zip': '60064-6400', 'sector': 'Healthcare', ..."
3,ACN,"{'zip': '2', 'sector': 'Technology', 'fullTime..."
4,AGN,"{'regularMarketPrice': None, 'logo_url': ''}"
...,...,...
59,UNP,"{'zip': '68179', 'sector': 'Industrials', 'ful..."
60,UPS,"{'zip': '30328', 'sector': 'Industrials', 'ful..."
61,USB,"{'zip': '55402', 'sector': 'Financial Services..."
62,RY.TO,"{'zip': 'M5J 2J5', 'sector': 'Financial Servic..."


In the next two cells, we filter out stocks that are either delisted, have an average daily volume of less than 10,000, or aren't US-listed. This is per the requirements of the assignment.

We also create a dict called hist_dict that stores the yf.Ticker(ticker).history() dataframe for each ticker. This is also for efficiency since calling .history() takes times for Yahoo to return the data, so it's faster to just store the data in memory than to call it each time we use the data - which we do a lot of since we make many calculations for our portfolio.

In [6]:
# Dict to store the ticker history for each ticker.
hist_dict = {}


def filter_us_market(df):
    """
    Consumes a dataframe of tickers and returns a list of booleans representing whether the tickers are US-listed or not.
    :param df: DataFrame containing tickers
    :return: List of booleans representing whether the tickers are US-listed.
    """
    # Initialize mask list of booleans.
    mask = []
    for idx, row in df.iterrows():
        # Check whether stock is US-listed or not, and append the boolean to mask
        info = row['Info']
        if "market" in row['Info']:
            is_us_market = info['market'] == 'us_market'
            mask.append(is_us_market)
        else:
            mask.append(False)
    # Return mask
    return mask


def filter_volume(df):
    """
    Consumes a dataframe of tickers and returns a list of bo
    :param df:
    :return:
    """
    # Start and end date to check volume
    start = "2021-07-02"
    end = "2021-10-23"
    # Mask to filter out stocks
    mask = []
    for idx, row in df.iterrows():
        # Gets ticker history
        ticker = yf.Ticker(row[0])
        ticker_hist = ticker.history(start=start_date, end=end_date)
        # Gets subset of data to check volume
        volume_hist = ticker_hist.loc[
            (ticker_hist.index >= pd.to_datetime(start)) & (ticker_hist.index <= pd.to_datetime(end))]
        # Checks if average volume is not less than 10,0000
        valid_volume = volume_hist['Volume'].mean() >= 10000
        mask.append(valid_volume)
        if valid_volume:
            hist_dict[row[0]] = ticker_hist  # Adds the ticker history dataframe to hist_dict
    return mask

In [7]:
# Filter out non-US stocks / delisted stocks
tickers_df = tickers_df.loc[filter_us_market(tickers_df)]
# Filter out stocks without valid volume
tickers_df = tickers_df.loc[filter_volume(tickers_df)]

- PCLN: No data found for this date range, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted


In [8]:
# Reset index on the filtered dataframe
tickers_df = tickers_df.reset_index(drop=True)
tickers_df

,Ticker,Info
0,AAPL,"{'zip': '95014', 'sector': 'Technology', 'full..."
1,ABBV,"{'zip': '60064-6400', 'sector': 'Healthcare', ..."
2,ABT,"{'zip': '60064-6400', 'sector': 'Healthcare', ..."
3,ACN,"{'zip': '2', 'sector': 'Technology', 'fullTime..."
4,AIG,"{'zip': '10020', 'sector': 'Financial Services..."
5,AMZN,"{'zip': '98109-5210', 'sector': 'Consumer Cycl..."
6,AXP,"{'zip': '10285', 'sector': 'Financial Services..."
7,BA,"{'zip': '60606-1596', 'sector': 'Industrials',..."
8,BAC,"{'zip': '28255', 'sector': 'Financial Services..."
9,BIIB,"{'zip': '02142', 'sector': 'Healthcare', 'full..."


Next, we reformatted the data such that it contains ticker history closing prices for the 1 year interval. For this data, we kept it as daily data rather than resampling it to monthly or weekly data.

The reason why we chose to use daily data to calculate the metrics for our portfolio (such as STD and Correlation) is because we found that it produced the best results when backtesting our short-term risky portfolio. We decided against using monthly closing prices for our portfolio since using 1 year of historical data would only give us 12 closing prices to decide our portfolio off of - which is too little. We also tried using weekly data and decided against it since we found that it didn't capture the most short-term volatile stocks. Weekly data tended to give us stocks that were volatile by coincidence (such as news causing a sharp drop or rise in stock price), rather stocks that are intrinsically volatile (like oil stocks, which are volatile not by coincidence but because oil is a volatile asset). Thus, we found that daily data gave us the most intrinsically short-term volatile stocks to build our risky portfolio.

In [9]:
def create_df(df):
    """
    This function iterates through the filtered list of stocks and creates a new dict with weekly ticker closing prices
    :param df: filtered stocks DataFrame
    :return: dict containing ticker closing prices resampled to weekly data
    """
    dic = {}
    for i in range(len(df)):
        tick = df.iloc[i, 0]
        # get ticker history data for the ticker
        t_hist = hist_dict[tick]
        # resample data to
        dic[tick] = t_hist.Close
    return dic


#creates the dataframe price using the function create_df
prices = pd.DataFrame(create_df(tickers_df))
prices

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,SLB,SO,SPG,T,TGT,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2020-11-27,115.875641,100.089195,105.981827,247.066132,38.660442,3195.340088,119.200394,216.500000,28.402325,243.779999,...,21.465225,58.201691,81.473923,27.017212,177.334900,155.335892,333.142670,202.732468,165.332474,42.830627
2020-11-30,118.320587,99.793381,106.572701,246.048706,37.424011,3168.040039,117.223442,210.710007,27.589148,240.169998,...,20.395889,57.433983,78.635902,26.756626,177.098145,157.543945,331.565399,201.194519,167.466568,41.861828
2020-12-01,121.968094,99.278099,106.838585,249.189896,38.271019,3220.080078,118.548004,213.009995,28.108404,243.160004,...,20.795614,58.124916,80.940590,26.868307,176.624664,159.732468,336.346527,199.538269,164.422089,42.869377
2020-12-02,122.325890,100.184608,106.503761,246.611755,38.679920,3203.530029,120.920349,223.850006,28.470903,246.919998,...,21.358191,59.382030,83.092918,27.073053,174.020416,158.980164,342.626099,200.681870,165.273758,43.285965
2020-12-03,122.186745,99.946053,105.893196,246.878433,38.787010,3186.729980,121.701241,237.199997,28.500479,244.089996,...,21.782593,59.382030,85.359520,27.203344,172.550598,158.257156,343.730194,199.400253,164.294815,43.566917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-18,157.869995,117.070000,127.639999,370.779999,56.119999,3696.060059,176.210007,227.250000,46.320000,256.790009,...,31.860001,61.630001,169.380005,24.389999,252.050003,193.470001,449.470001,240.240005,206.449997,58.650002
2021-11-19,160.550003,116.239998,126.839996,369.790009,54.950001,3676.570068,173.539993,214.130005,45.400002,257.190002,...,30.049999,62.669998,166.740005,24.129999,250.699997,195.160004,440.000000,241.309998,208.470001,57.590000
2021-11-22,161.020004,115.650002,125.260002,363.299988,56.939999,3572.570068,170.899994,209.899994,46.279999,252.210007,...,30.340000,63.119999,167.990005,24.700001,244.570007,192.500000,437.540009,243.910004,210.880005,58.980000


In the next 2 cells, we obtain the stock with the highest risk (calculated using standard deviation). This will be a key part in forming our portfolio, as explained in the next annotation cell.

In [10]:
def generate_std(df):
    """
    Creates a dict of tickers and their standard deviations for the given closing prices
    :param df: Filtered DataFrame of tickers
    :return: Dict of tickers and their STDs
    """
    column = df.columns
    stdlst_dic = {}
    # Calculate the STD for each ticker
    for i in range(len(column)):
        temp = df[str(column[i])].pct_change().std()
        stdlst_dic[str(column[i])] = temp
    return stdlst_dic


#creates the dataframe slst which is a dictionary containing the tickers and their std
slst = generate_std(prices)
slst

{'AAPL': 0.015589216108291292,
 'ABBV': 0.012687257124244495,
 'ABT': 0.012937698430129707,
 'ACN': 0.011662958326867374,
 'AIG': 0.018063390358586,
 'AMZN': 0.015003883728443813,
 'AXP': 0.01616605643958101,
 'BA': 0.021778160726516737,
 'BAC': 0.015898391225207427,
 'BIIB': 0.029952123725821768,
 'BK': 0.015778106612080692,
 'BLK': 0.013975514658661896,
 'BMY': 0.011256806383796262,
 'C': 0.01668873237382216,
 'CAT': 0.015894093842269762,
 'CL': 0.008721513148153833,
 'CMCSA': 0.014581208484893506,
 'COF': 0.019465250219560925,
 'COP': 0.023323831699250588,
 'COST': 0.010578528940163602,
 'CSCO': 0.011399690446090375,
 'CVS': 0.01346667763553167,
 'GM': 0.023305596804548735,
 'GOOG': 0.014676421421111563,
 'JPM': 0.013588242909452859,
 'KMI': 0.01575098955162709,
 'KO': 0.009007814340040934,
 'LLY': 0.019423620273193422,
 'LMT': 0.012552751752187539,
 'MO': 0.013537004194831535,
 'MON': 0.004634406274978367,
 'MRK': 0.014224658706541595,
 'MS': 0.016524613011823067,
 'MSFT': 0.012480

In [11]:
#function to find the highest std
def get_highest_std(dic):
    """
    Consumes a dictionary of tickers and their STDs and returns the ticker with the highest STD in the dict
    :param dic: Dict of tickers and their STDs
    :return: Ticker with the highest STD
    """
    std = 0
    tick = ""
    # Calculates the highest STD
    for i in dic:
        if std < dic[i]:
            std = dic[i]
            tick = i
    return tick


#checking what the highest std is
get_highest_std(slst)

'OXY'

Next we'll generate a correlation matrix for all the stocks

In [12]:
#creating a correlation dataframe
corr_df = prices.corr()
corr_df

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,SLB,SO,SPG,T,TGT,TXN,UNH,UNP,UPS,USB
AAPL,1.000000,0.604065,0.664993,0.861126,0.614020,0.738855,0.637127,-0.427252,0.542324,0.301457,...,0.278684,0.575481,0.675865,-0.619940,0.799600,0.630674,0.703270,0.446045,0.497475,0.437071
ABBV,0.604065,1.000000,0.412300,0.702714,0.659321,0.587217,0.780943,0.034186,0.698012,0.562645,...,0.631552,0.690342,0.742411,-0.155989,0.825548,0.636738,0.820446,0.725604,0.816459,0.688787
ABT,0.664993,0.412300,1.000000,0.691319,0.609786,0.414785,0.532036,-0.224950,0.583567,0.095945,...,0.289304,0.501690,0.651632,-0.335467,0.566345,0.624754,0.543077,0.458720,0.305133,0.481350
ACN,0.861126,0.702714,0.691319,1.000000,0.885146,0.636466,0.867641,-0.151026,0.837334,0.280985,...,0.587046,0.736482,0.896635,-0.543035,0.896136,0.815421,0.917413,0.672664,0.699038,0.761137
AIG,0.614020,0.659321,0.609786,0.885146,1.000000,0.374889,0.902364,0.064450,0.957448,0.281582,...,0.797476,0.732990,0.928886,-0.380366,0.801239,0.837272,0.870127,0.689875,0.692909,0.903536
AMZN,0.738855,0.587217,0.414785,0.636466,0.374889,1.000000,0.587539,-0.130097,0.425675,0.430400,...,0.326677,0.458268,0.560117,-0.307654,0.702175,0.580967,0.612062,0.422127,0.635151,0.393448
AXP,0.637127,0.780943,0.532036,0.867641,0.902364,0.587539,1.000000,0.170938,0.919203,0.496619,...,0.831438,0.713226,0.924863,-0.293261,0.910207,0.889941,0.902144,0.692615,0.809593,0.913931
BA,-0.427252,0.034186,-0.224950,-0.151026,0.064450,-0.130097,0.170938,1.000000,0.186698,0.204948,...,0.295437,0.117328,0.087042,0.507893,-0.041560,0.196403,0.028462,0.177976,0.133089,0.304110
BAC,0.542324,0.698012,0.583567,0.837334,0.957448,0.425675,0.919203,0.186698,1.000000,0.307045,...,0.871160,0.692120,0.941351,-0.270875,0.795095,0.868130,0.883567,0.799033,0.758680,0.958982
BIIB,0.301457,0.562645,0.095945,0.280985,0.281582,0.430400,0.496619,0.204948,0.307045,1.000000,...,0.433273,0.425073,0.352118,0.032410,0.586910,0.453452,0.365653,0.181883,0.457321,0.338984


In the next cells, we use correlation to calculate stocks that trade in the same direction as the stock with the highest STD, in order to create a portfolio where all the stocks are strongly positively correlated with each other. In our original code, we instead compiled a list of stocks in the same industry to achieve the opposite affect of inter-industry diversification. However, we then realized that it would make more sense to choose a basket stocks that are strongly positively correlated with each other, since it would acheive the same effect as an intra-industry portfolio, but would give us a larger variety of risky stocks to choose from so we can choose the riskiest of stocks.

In [13]:
def lister(corr, highest_std):
    """
    Creates a dict of stocks that are either positively or negatively correlated with the highest std stock
    :param corr: Correlation matrix of all stocks
    :param highest_std: Ticker of stock with the highest
    :return:
    """
    list1 = []
    list2 = []
    columns = corr.columns
    for i in range(len(corr.index)):
        c = corr[highest_std].iloc[i]
        tick = columns[i]
        if c > 0:
            list1.append(tick)
        elif c < 0:
            list2.append(tick)
    dic = {"positive": list1, "negative": list2}
    return dic


#creates dictionary cor_list which calls the function lister
# contains two lists which are positive correlating and negative correlating with the inputed stock
cor_list = lister(corr_df, get_highest_std(slst))
cor_list

{'positive': ['AAPL',
  'ABBV',
  'ABT',
  'ACN',
  'AIG',
  'AMZN',
  'AXP',
  'BA',
  'BAC',
  'BIIB',
  'BK',
  'BLK',
  'C',
  'CAT',
  'CMCSA',
  'COF',
  'COP',
  'COST',
  'CSCO',
  'CVS',
  'GM',
  'GOOG',
  'JPM',
  'KMI',
  'KO',
  'LLY',
  'LMT',
  'MO',
  'MON',
  'MRK',
  'MS',
  'MSFT',
  'NEE',
  'NKE',
  'ORCL',
  'OXY',
  'PEP',
  'PFE',
  'PG',
  'PM',
  'PYPL',
  'SBUX',
  'SLB',
  'SO',
  'SPG',
  'TGT',
  'TXN',
  'UNH',
  'UNP',
  'UPS',
  'USB'],
 'negative': ['BMY', 'CL', 'QCOM', 'T']}

We then sort the both baskets of positively and negatively correlated stocks by their STDs in descending order so we can choose stocks that have the highest STD and are also strongly positively correlated with our keystone stock - the stock that has the highest std and weighting in our final portfolio.

In [14]:
def quicksort(tick_lst, std_list):
    """
    Implementation of quicksort to sort a list of tickers based on their std
    :param tick_lst: Ticker list
    :param std_list: List of respective STDs of the tickers.
    :return: Sorted list of tickers
    """
    sequence = tick_lst
    length = len(sequence)
    if length <= 1:
        return sequence
    else:
        pivot = sequence.pop()

    items_greater = []
    items_lower = []

    for item in sequence:
        i = std_list[item]
        pivoter = std_list[pivot]
        if i < pivoter:
            items_lower.append(item)
        else:
            items_greater.append(item)

    return quicksort(items_greater, slst) + [pivot] + quicksort(items_lower, slst)


positive = quicksort(cor_list["positive"], slst)
negative = (quicksort(cor_list["negative"], slst))

positive

['OXY',
 'BIIB',
 'SLB',
 'COP',
 'GM',
 'PYPL',
 'BA',
 'SPG',
 'COF',
 'LLY',
 'AIG',
 'NKE',
 'C',
 'MS',
 'TXN',
 'USB',
 'AXP',
 'BAC',
 'CAT',
 'BK',
 'KMI',
 'AAPL',
 'UPS',
 'PFE',
 'AMZN',
 'GOOG',
 'CMCSA',
 'ORCL',
 'MRK',
 'TGT',
 'SBUX',
 'BLK',
 'NEE',
 'JPM',
 'MO',
 'CVS',
 'ABT',
 'UNP',
 'ABBV',
 'LMT',
 'MSFT',
 'UNH',
 'ACN',
 'CSCO',
 'PM',
 'COST',
 'SO',
 'KO',
 'PEP',
 'PG',
 'MON']

We also have a function to ensure that our code doesn't acheive the opposite of a risky portfolio - in the case that there is less than 10 stocks that are positively correlated with the stock with the highest STD, it might choose stocks that are negatively correlated and create a balanced portfolio. To overcome this, we make sure that the STD of the basket of stocks WITH the highest correlation is higher than the STD of the basket of stocks that are strongly negatively correlated against the stock with the highest STD. In the case that this is false, then our portfolio becomes the stocks with the strongly negatively correlated stocks. In either case, this ensures that our portfolio will have the highest STD possible, and all the stocks in the portfolio trade in the same direction.

In [15]:
def meanstd(lst, count, std_list, sum, lst_tick):
    """
    Recursively calculates the average STD of the top 10 stocks in each list of positive and negative correlated stocks.
    :param lst: List of tickers
    :param count: Iterative counter
    :param std_list: Ticker's stds
    :param sum: Sum of the STD
    :param lst_tick: Second list of tickers
    :return: Average STD of stocks in a list
    """
    if 0 == len(lst):
        return {"list": lst_tick, "mean": (sum / count)}
    elif 10 == count:
        return {"list": lst_tick, "mean": (sum / 10)}
    else:
        sum = sum + std_list[lst[0]]
        lst_tick.append(lst[0])
        return meanstd(lst[1:], (count + 1), std_list, sum, lst_tick)


pm = meanstd(positive, 0, slst, 0, [])
nm = meanstd(negative, 0, slst, 0, [])


def hstd(p, n):
    """
    Gets the higher average std of the positively and negatively correlated stocks
    :param p: Positively correlated stock std
    :param n: Negatively correlated stock std
    :return: Higher average STD
    """
    if p["mean"] > n["mean"] and len(p["list"]) >= 7:
        p["oppo"] = negative
        return p
    elif n["mean"] > p["mean"] and len(n["list"]) >= 7:
        n["oppo"] = positive
        return n


port_list = hstd(pm, nm)
port_list

{'list': ['OXY',
  'BIIB',
  'SLB',
  'COP',
  'GM',
  'PYPL',
  'BA',
  'SPG',
  'COF',
  'LLY'],
 'mean': 0.02446219897854984,
 'oppo': ['QCOM', 'BMY', 'T', 'CL']}

In the case that the list of tickers given to us is less than 20 (such as 6 positively correlated stocks and 5 negatively correlated stocks), our portfolio appends the negatively correlated stocks with the lowest STD to our positively correlated portfolio. This ensures that our portfolio has at least 10 stocks and the negatively correlated stocks do not affect our portfolio performance as much.

In [16]:
def portlength(port):
    """
    Appends weakly negatively correlated stocks to our portfolio if there aren't enough positively correlated stocks in our portfolio.
    :param port: Portfolio tickers
    :return: New portfolio
    """
    if len(port["list"]) == 10:
        return port["list"]
    else:
        for i in range(1, 10 - len(port)):
            if i == 10:
                return port["list"]
            port["list"].append(port["oppo"][-1 * i])


finalport_lst = portlength(port_list)
finalport_lst

['OXY', 'BIIB', 'SLB', 'COP', 'GM', 'PYPL', 'BA', 'SPG', 'COF', 'LLY']

In [17]:
lstofstocks = finalport_lst

Finally, we set our weights in this cell and backtest our portfolio over 2021-11-01 to 2021-11-26. For our weightings, we made them

35% 25% 5% 5% 5% 5% 5% 5% 5% 5%

Where the stock with the highest STD in our portfolio is weighted at 35% and the stock with the lowest STD is weighted at 5%. This ensures that overall, our portfolio has the highest STD as allowed by the requirements of the assignment (limited by the weightings), by giving the highest weighting to the riskiest stock. That way, a volatile move by the stock with the highest std will affect our portfolio the most.

In [18]:
# Using the 35% distribution

startdate = '2021-11-01'
enddate = '2021-11-26'

#Determine number of shares bought for each stocks under 35% weighted profolio ($100,000)
# Assuming that the stocks i the list are given in increasing order
NumOfShares = []  # number of shares will be printed in the list according to the corrosponding order


# Loopsahre function takes a list the starting date and the ending date, and produces the number of shares for each stocks in the list given
# Since the stocks given in the list is already sorted in decreasing order of standard deviation, we con proceed to apply the
# 35% percent distribution method (explained in the report)
def loopshare(lst, date_start, date_end):
    for i in range(len(lst)):
        if (i < 1):
            NumOfShares.append(35000 / ((yf.Ticker(lst[i]).history(start=date_start, end=date_end)).iloc[0, 3]))
        elif (i < 2):
            NumOfShares.append(25000 / ((yf.Ticker(lst[i]).history(start=date_start, end=date_end)).iloc[0, 3]))
        else:
            NumOfShares.append(5000 / ((yf.Ticker(lst[i]).history(start=date_start, end=date_end)).iloc[0, 3]))
    return NumOfShares


loopshare(lstofstocks, startdate, enddate)

listOfClose = []  #list of dataframe


#Extracting the closed data

# Funcation loopClose takes a lst of tickers, start date, and a end date, and produces a list of closing price of each stocks in the list date
# Withtin the start and end date
def loopClose(lst, start_date, end_date):
    for i in range(len(lst)):
        listOfClose.append(pd.DataFrame((yf.Ticker(lst[i]).history(start=start_date, end=end_date))['Close']))


loopClose(lstofstocks, startdate, enddate)

#Setting up a list before adding it to the dataframe
listBefore = []


#Function loopCloseBefore, creates a list of closing prices of tickers that is ready to be put into a dataframe
def loopCloseBefore(lst):
    for i in range(len(lst)):
        if (i == 0):
            listBefore.append(lst[i])
        else:
            listBefore.append(lst[i]['Close'])


loopCloseBefore(listOfClose)

#Create the dataframe for tracking the portfolio

#concating all the list of closign prices of stocks
finalPortfolio = pd.concat(listBefore, join='inner', axis=1)

#Renaming the column names to the corrosponding stocks
finalPortfolio.columns = lstofstocks

#Adding portfolio

#Adding the column portfolio
finalPortfolio['portfolio'] = finalPortfolio[str(finalPortfolio.columns[0])] * NumOfShares[0] + finalPortfolio[
    str(finalPortfolio.columns[1])] * NumOfShares[1] + finalPortfolio[str(finalPortfolio.columns[2])] * NumOfShares[2] + \
                              finalPortfolio[str(finalPortfolio.columns[3])] * NumOfShares[3] + finalPortfolio[
                                  str(finalPortfolio.columns[4])] * NumOfShares[4] + finalPortfolio[
                                  str(finalPortfolio.columns[5])] * NumOfShares[5] + finalPortfolio[
                                  str(finalPortfolio.columns[6])] * NumOfShares[6] + finalPortfolio[
                                  str(finalPortfolio.columns[7])] * NumOfShares[7] + finalPortfolio[
                                  str(finalPortfolio.columns[8])] * NumOfShares[8] + finalPortfolio[
                                  str(finalPortfolio.columns[9])] * NumOfShares[9]

As seen below, over the course of 1 month from 2021-11-01 to 2021-11-26, our portfolio fell 5%. At the lowest, it fell nearly 8%. The backtesting of our portfolio on historical data reveals that it is very risky as desired.

In [19]:
finalPortfolio

,OXY,BIIB,SLB,COP,GM,PYPL,BA,SPG,COF,LLY,portfolio
Date,,,,,,,,,,,
2021-11-01,34.810001,272.100006,33.299999,74.589996,55.490002,231.279999,214.580002,149.360001,152.425797,248.503113,100000.000000
2021-11-02,34.360001,272.510010,33.200001,73.029999,55.590000,229.460007,212.770004,158.990005,151.330002,256.018707,99830.763905
2021-11-03,33.849998,279.589996,32.980000,72.739998,56.000000,230.380005,213.380005,166.240005,153.312393,265.398254,100483.497975
2021-11-04,33.750000,280.410004,33.150002,73.309998,58.639999,228.220001,213.029999,165.710007,152.874084,270.122925,100767.999797
2021-11-05,33.419998,278.679993,33.509998,75.449997,58.520000,225.779999,224.460007,170.500000,154.278702,264.142334,100763.616773
2021-11-08,34.290001,272.700012,34.299999,75.930000,59.599998,229.419998,222.679993,167.389999,154.926224,259.686798,101201.747082
2021-11-09,33.740002,270.029999,34.139999,75.750000,58.619999,205.419998,220.789993,167.490005,154.826599,261.171967,99746.107295
2021-11-10,31.820000,266.929993,32.730000,73.000000,59.270000,204.639999,218.500000,163.970001,155.683319,263.793457,97086.109490
2021-11-11,32.119999,266.570007,32.779999,72.139999,61.820000,202.029999,219.380005,165.440002,156.549988,261.779999,97535.509001


In [20]:
def price_data(lst,date):
    dic={}
    for i in range(len(lst)):
        t=yf.Ticker(lst[i])
        t_hist=t.history(start=date)
        dic[lst[i]]=t_hist.Close
    return dic

comp_closing=pd.DataFrame(price_data(lstofstocks, comp_date))

def calc_shares(df):
    dic={}
    for i in range(len(df.index)):
        comp_closing.index[i]
        for j in range(len(df.columns)):
            if j <= 0:
                dic[df.columns[j]]=35000/df.iloc[i,j]
            elif j <= 1:
                dic[df.columns[j]]=25000/df.iloc[i,j]
            else:
                dic[df.columns[j]]=5000/df.iloc[i,j]
    return dic

shares=calc_shares(comp_closing)
sa=pd.DataFrame.from_dict(shares,orient="index")

comp_closing=comp_closing.transpose()
FinalPortfolio=comp_closing

FinalPortfolio["Shares"]=sa[0]
FinalPortfolio=comp_closing.reset_index()
FinalPortfolio.columns=["Ticker","Prices","Shares"]
FinalPortfolio["Values"]=FinalPortfolio["Prices"]*FinalPortfolio["Shares"]
FinalPortfolio["Weight"]=(FinalPortfolio["Values"]/100000)*100
FinalPortfolio

,Ticker,Prices,Shares,Values,Weight
0,OXY,32.009998,1093.408367,35000.0,35.0
1,BIIB,250.130005,99.948025,25000.0,25.0
2,SLB,31.410000,159.184974,5000.0,5.0
3,COP,74.830002,66.818119,5000.0,5.0
4,GM,62.189999,80.398780,5000.0,5.0
5,PYPL,188.710007,26.495680,5000.0,5.0
6,BA,210.600006,23.741690,5000.0,5.0
7,SPG,169.029999,29.580548,5000.0,5.0
8,COF,155.860001,32.080072,5000.0,5.0
9,LLY,262.000000,19.083969,5000.0,5.0


In [21]:
Stocks=pd.DataFrame(FinalPortfolio["Ticker"])
Stocks["Shares"]=FinalPortfolio["Shares"]
Stocks

,Ticker,Shares
0,OXY,1093.408367
1,BIIB,99.948025
2,SLB,159.184974
3,COP,66.818119
4,GM,80.398780
5,PYPL,26.495680
6,BA,23.741690
7,SPG,29.580548
8,COF,32.080072
9,LLY,19.083969


In [23]:
Stocks.to_csv('Stocks_Group_12.csv', index=False)
pd.read_csv('Stocks_Group_12.csv')

,Ticker,Shares
0,OXY,1093.408367
1,BIIB,99.948025
2,SLB,159.184974
3,COP,66.818119
4,GM,80.398780
5,PYPL,26.495680
6,BA,23.741690
7,SPG,29.580548
8,COF,32.080072
9,LLY,19.083969


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.